In [8]:
import json
from langchain.docstore.document import Document
import glob
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from semantic_rag import SemanticJSONSplitter, load_and_process_documents

In [9]:
directory_path = './OKWs/'
documents = load_and_process_documents(directory_path)


TypeError: Can't instantiate abstract class SemanticJSONSplitter without an implementation for abstract method 'split_text'

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 30
    )

splits = text_splitter.split_documents(documents)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})


/var/folders/hh/n0z7fj213rs0qtpsswcnxxl40000gp/T/ipykernel_21863/1855002374.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
question = 'Is there any bakery near me that can give me organic flour? Or that produces fermented bread.'

In [8]:
initial_docs = retriever.get_relevant_documents(question)
documents_texts = [doc.page_content for doc in initial_docs]

data = {
    'model': 'jina-reranker-v2-base-en',
    'query': question,
    'top_n': 5,
    'documents': documents_texts
}


/var/folders/hh/n0z7fj213rs0qtpsswcnxxl40000gp/T/ipykernel_21863/2071617569.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_docs = retriever.get_relevant_documents(question)


In [9]:
import requests

url = 'https://api.jina.ai/v1/rerank'
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer key_here'
}

response = requests.post(url, headers=headers, json=data)
rerank_results = response.json()



In [10]:
# Process the reranker response
if 'results' in rerank_results:
    reranked_docs = []
    for result in rerank_results['results']:
        index = result['index']
        reranked_doc = initial_docs[index]
        reranked_docs.append(reranked_doc)
else:
    # Handle error
    print("Error in reranking:", rerank_results)
    reranked_docs = initial_docs  # Fallback to initial docs

Error in reranking: {'detail': 'Model jina-reranker-v2-base-en not found'}


In [11]:
# Option 1: Simple list of entities
entities = []
for doc in reranked_docs:
    title = doc.metadata.get('title', 'No Title')
    description = doc.page_content
    entities.append({'title': title, 'description': description})

print("Entities matching your requirements:")
for entity in entities:
    print(f"Title: {entity['title']}")
    print(f"Description: {entity['description']}\n")


Entities matching your requirements:
Title: Bread & Butter Bakery
Description: Keywords: bakery, artisanal bread, pastries, local ingredients
Inventory Atoms: Organic Flour: Flour milled from organically grown grains.

Title: Bread & Butter Bakery
Description: Title: Bread & Butter Bakery
Description: A bakery specializing in sourdough bread and handcrafted pastries using locally sourced ingredients.

Title: Bread & Butter Bakery
Description: Product Atoms: Sourdough Loaves: Bread made through natural fermentation.
Tool List Atoms: Stone Oven: Traditional stone oven for baking.
Processes: Baking, Fermentation

Title: SkyGrow Hydroponics
Description: Tool List Atoms: Hydroponic Systems: Equipment for soilless cultivation.
Processes: Soilless Cultivation, Vertical Farming

Title: EcoEnergy Solutions
Description: Title: EcoEnergy Solutions
Description: Provides solar panel installation and wind turbine services for residential and commercial clients.

